# AzureOpenAIEmbeddings

[Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service/) is a cloud service to help you quickly develop generative AI experiences with a diverse set of prebuilt and curated models from OpenAI, Meta and beyond.

LangChain.js supports integration with [Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service/) using the new Azure integration in the [OpenAI SDK](https://github.com/openai/openai-node).

You can learn more about Azure OpenAI and its difference with the OpenAI API on [this page](https://learn.microsoft.com/azure/ai-services/openai/overview). If you don't have an Azure account, you can [create a free account](https://azure.microsoft.com/free/) to get started.

This will help you get started with AzureOpenAIEmbeddings [embedding models](/docs/concepts/embedding_models) using LangChain. For detailed documentation on `AzureOpenAIEmbeddings` features and configuration options, please refer to the [API reference](https://api.js.langchain.com/classes/langchain_openai.AzureOpenAIEmbeddings.html).


```{=mdx}

:::info

Previously, LangChain.js supported integration with Azure OpenAI using the dedicated [Azure OpenAI SDK](https://github.com/Azure/azure-sdk-for-js/tree/main/sdk/openai/openai). This SDK is now deprecated in favor of the new Azure integration in the OpenAI SDK, which allows to access the latest OpenAI models and features the same day they are released, and allows seamless transition between the OpenAI API and Azure OpenAI.

If you are using Azure OpenAI with the deprecated SDK, see the [migration guide](#migration-from-azure-openai-sdk) to update to the new API.

:::

```

## Overview
### Integration details

| Class | Package | Local | [Py support](https://python.langchain.com/docs/integrations/text_embedding/azureopenai/) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [AzureOpenAIEmbeddings](https://api.js.langchain.com/classes/langchain_openai.AzureOpenAIEmbeddings.html) | [@langchain/openai](https://api.js.langchain.com/modules/langchain_openai.html) | ❌ | ✅ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/openai?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/openai?style=flat-square&label=%20&) |

## Setup

To access Azure OpenAI embedding models you'll need to create an Azure account, get an API key, and install the `@langchain/openai` integration package.

### Credentials

You'll need to have an Azure OpenAI instance deployed. You can deploy a version on Azure Portal following [this guide](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal).

Once you have your instance running, make sure you have the name of your instance and key. You can find the key in the Azure Portal, under the "Keys and Endpoint" section of your instance.

If you're using Node.js, you can define the following environment variables to use the service:

```bash
AZURE_OPENAI_API_INSTANCE_NAME=<YOUR_INSTANCE_NAME>
AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME=<YOUR_EMBEDDINGS_DEPLOYMENT_NAME>
AZURE_OPENAI_API_KEY=<YOUR_KEY>
AZURE_OPENAI_API_VERSION="2024-02-01"
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```bash
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="your-api-key"
```

### Installation

The LangChain AzureOpenAIEmbeddings integration lives in the `@langchain/openai` package:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/openai @langchain/core
</Npm2Yarn>

:::info

You can find the list of supported API versions in the [Azure OpenAI documentation](https://learn.microsoft.com/azure/ai-services/openai/reference).

:::

:::tip

If `AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME` is not defined, it will fall back to the value of `AZURE_OPENAI_API_DEPLOYMENT_NAME` for the deployment name. The same applies to the `azureOpenAIApiEmbeddingsDeploymentName` parameter in the `AzureOpenAIEmbeddings` constructor, which will fall back to the value of `azureOpenAIApiDeploymentName` if not defined.

:::

```

## Instantiation

Now we can instantiate our model object and embed text:

In [1]:
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const embeddings = new AzureOpenAIEmbeddings({
  azureOpenAIApiKey: "<your_key>", // In Node.js defaults to process.env.AZURE_OPENAI_API_KEY
  azureOpenAIApiInstanceName: "<your_instance_name>", // In Node.js defaults to process.env.AZURE_OPENAI_API_INSTANCE_NAME
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embeddings_deployment_name>", // In Node.js defaults to process.env.AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME
  azureOpenAIApiVersion: "<api_version>", // In Node.js defaults to process.env.AZURE_OPENAI_API_VERSION
  maxRetries: 1,
});

## Indexing and Retrieval

Embedding models are often used in retrieval-augmented generation (RAG) flows, both as part of indexing data as well as later retrieving it. For more detailed instructions, please see our RAG tutorials under the [working with external knowledge tutorials](/docs/tutorials/#working-with-external-knowledge).

Below, see how to index and retrieve data using the `embeddings` object we initialized above. In this example, we will index and retrieve a sample document using the demo [`MemoryVectorStore`](/docs/integrations/vectorstores/memory).

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## Direct Usage

Under the hood, the vectorstore and retriever implementations are calling `embeddings.embedDocument(...)` and `embeddings.embedQuery(...)` to create embeddings for the text(s) used in `fromDocuments` and the retriever's `invoke` operations, respectively.

You can directly call these methods to get embeddings for your own use cases.

### Embed single texts

You can embed queries for search with `embedQuery`. This generates a vector representation specific to the query:

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
   -0.024253517, -0.0054218727,   0.048715446,   0.020580322,    0.03180832,
   0.0028770117,  -0.012367731,   0.037383243,  -0.054915592,   0.032225136,
     0.00825818,  -0.023888804,   -0.01184671,   0.012257014,   0.016294925,
    0.009254632,  0.0051353113,  -0.008889917,   0.016855022,    0.04207243,
  0.00082589936,  -0.011664353,    0.00818654,   0.029020859,  -0.012335167,
   -0.019603407,  0.0013945447,    0.05538451,  -0.011625277,  -0.008153976,
    0.038607642,   -0.03811267, -0.0074440846,   0.047647353,   -0.00927417,
    0.024201415, -0.0069230637,  -0.008538228,   0.003910912,   0.052805457,
   -0.023159374,  0.0014352495,  -0.038659744,   0.017141584,   0.005587948,
    0.007971618,  -0.016920151,    0.06658646, -0.0016916894,   0.045667473,
   -0.042202685,   -0.03983204,   -0.04160351,  -0.011729481,  -0.055905532,
    0.012543576,  0.0038848612,   0.007919516,   0.010915386,  0.0033117384,
   -0.007548289,  -0.030427614,  -0.041890074,   0.036002535,  -0.02377157

### Embed multiple texts

You can embed multiple texts for indexing with `embedDocuments`. The internals used for this method may (but do not have to) differ from embedding queries:

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
   -0.024253517, -0.0054218727,   0.048715446,   0.020580322,    0.03180832,
   0.0028770117,  -0.012367731,   0.037383243,  -0.054915592,   0.032225136,
     0.00825818,  -0.023888804,   -0.01184671,   0.012257014,   0.016294925,
    0.009254632,  0.0051353113,  -0.008889917,   0.016855022,    0.04207243,
  0.00082589936,  -0.011664353,    0.00818654,   0.029020859,  -0.012335167,
   -0.019603407,  0.0013945447,    0.05538451,  -0.011625277,  -0.008153976,
    0.038607642,   -0.03811267, -0.0074440846,   0.047647353,   -0.00927417,
    0.024201415, -0.0069230637,  -0.008538228,   0.003910912,   0.052805457,
   -0.023159374,  0.0014352495,  -0.038659744,   0.017141584,   0.005587948,
    0.007971618,  -0.016920151,    0.06658646, -0.0016916894,   0.045667473,
   -0.042202685,   -0.03983204,   -0.04160351,  -0.011729481,  -0.055905532,
    0.012543576,  0.0038848612,   0.007919516,   0.010915386,  0.0033117384,
   -0.007548289,  -0.030427614,  -0.041890074,   0.036002535,  -0.02377157

## Using Azure Managed Identity

If you're using Azure Managed Identity, you can configure the credentials like this:

In [ ]:
import {
  DefaultAzureCredential,
  getBearerTokenProvider,
} from "@azure/identity";
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const credentials = new DefaultAzureCredential();
const azureADTokenProvider = getBearerTokenProvider(
  credentials,
  "https://cognitiveservices.azure.com/.default"
);

const modelWithManagedIdentity = new AzureOpenAIEmbeddings({
  azureADTokenProvider,
  azureOpenAIApiInstanceName: "<your_instance_name>",
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embeddings_deployment_name>",
  azureOpenAIApiVersion: "<api_version>",
});


## Using a different domain

If your instance is hosted under a domain other than the default `openai.azure.com`, you'll need to use the alternate `AZURE_OPENAI_BASE_PATH` environment variable.
For example, here's how you would connect to the domain `https://westeurope.api.microsoft.com/openai/deployments/{DEPLOYMENT_NAME}`:

In [ ]:
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const embeddingsDifferentDomain = new AzureOpenAIEmbeddings({
  azureOpenAIApiKey: "<your_key>", // In Node.js defaults to process.env.AZURE_OPENAI_API_KEY
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embedding_deployment_name>", // In Node.js defaults to process.env.AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME
  azureOpenAIApiVersion: "<api_version>", // In Node.js defaults to process.env.AZURE_OPENAI_API_VERSION
  azureOpenAIBasePath:
    "https://westeurope.api.microsoft.com/openai/deployments", // In Node.js defaults to process.env.AZURE_OPENAI_BASE_PATH
});


## Custom headers

You can specify custom headers by passing in a `configuration` field:

In [ ]:
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const embeddingsWithCustomHeaders = new AzureOpenAIEmbeddings({
  azureOpenAIApiKey: "<your_key>",
  azureOpenAIApiInstanceName: "<your_instance_name>",
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embeddings_deployment_name>",
  azureOpenAIApiVersion: "<api_version>",
  configuration: {
    defaultHeaders: {
      "x-custom-header": `SOME_VALUE`,
    },
  },
});

The `configuration` field also accepts other `ClientOptions` parameters accepted by the official SDK.

**Note:** The specific header `api-key` currently cannot be overridden in this manner and will pass through the value from `azureOpenAIApiKey`.

## Migration from Azure OpenAI SDK

If you are using the deprecated Azure OpenAI SDK with the `@langchain/azure-openai` package, you can update your code to use the new Azure integration following these steps:

1. Install the new `@langchain/openai` package and remove the previous `@langchain/azure-openai` package:
   ```bash npm2yarn
   npm install @langchain/openai
   npm uninstall @langchain/azure-openai
   ```
2. Update your imports to use the new `AzureOpenAIEmbeddings` classe from the `@langchain/openai` package:
   ```typescript
   import { AzureOpenAIEmbeddings } from "@langchain/openai";
   ```
3. Update your code to use the new `AzureOpenAIEmbeddings` class and pass the required parameters:

   ```typescript
   const model = new AzureOpenAIEmbeddings({
     azureOpenAIApiKey: "<your_key>",
     azureOpenAIApiInstanceName: "<your_instance_name>",
     azureOpenAIApiEmbeddingsDeploymentName:
       "<your_embeddings_deployment_name>",
     azureOpenAIApiVersion: "<api_version>",
   });
   ```

   Notice that the constructor now requires the `azureOpenAIApiInstanceName` parameter instead of the `azureOpenAIEndpoint` parameter, and adds the `azureOpenAIApiVersion` parameter to specify the API version.

   - If you were using Azure Managed Identity, you now need to use the `azureADTokenProvider` parameter to the constructor instead of `credentials`, see the [Azure Managed Identity](#using-azure-managed-identity) section for more details.

   - If you were using environment variables, you now have to set the `AZURE_OPENAI_API_INSTANCE_NAME` environment variable instead of `AZURE_OPENAI_API_ENDPOINT`, and add the `AZURE_OPENAI_API_VERSION` environment variable to specify the API version.


## API reference

For detailed documentation of all AzureOpenAIEmbeddings features and configurations head to the API reference: https://api.js.langchain.com/classes/langchain_openai.AzureOpenAIEmbeddings.html